In [12]:
import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, Dropout, Flatten
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard

In [27]:
file = open("nietzsche.txt", encoding="utf-8")
text_niet = file.read()
# 732020:732176

file_date = open("baddate.txt", encoding="utf-8")
text_date = file_date.read()

EPOCHS = 10
diversity = 0.5

# Do we not
# hear the noise of the grave-diggers who are burying God? Do we not smell
# the divine putrefaction?—for even Gods putrefy! God is dead! God remains
# dead! And we have killed him!


In [20]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)
    preds = preds / np.sum(preds)               
    probas = np.random.multinomial(1, preds, 1) 
    return np.argmax(probas)
    


In [16]:
chars = sorted(list(set(text_niet)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 40
step = seqlen
sentences = []
for i in range(40, len(text_niet) - seqlen - 1, step):
    sentences.append(text_niet[i: i + seqlen + 1])

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1
        
model1 = Sequential()
model1.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model1.add(Dense(len(chars), activation='softmax'))

model1.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def on_epoch_end_niet(epoch, _):
    """Function invoked at end of each epoch."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text_niet) - seqlen - 1)

    generated = ''
    sentence = text_niet[start_index: start_index + seqlen]
    generated += sentence
    #print('----- Generating with seed: "' + sentence + '"')
    #sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
        
        preds = model1.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        #sys.stdout.write(next_char) 

    # for diversity in [0.2, 0.5, 1.0]:
    #     #print('----- diversity:', diversity)

    #     generated = ''
    #     sentence = text_niet[start_index: start_index + seqlen]
    #     generated += sentence
    #     #print('----- Generating with seed: "' + sentence + '"')
    #     #sys.stdout.write(generated)

    #     for i in range(400):
    #         x_pred = np.zeros((1, seqlen, len(chars)))
    #         for t, char in enumerate(sentence):
    #             x_pred[0, t, char_indices[char]] = 1.
            
    #         preds = model1.predict(x_pred, verbose=0)
    #         next_index = sample(preds[0, -1], diversity)
    #         next_char = indices_char[next_index]

    #         sentence = sentence[1:] + next_char

    #         #sys.stdout.write(next_char)   

print_callback = LambdaCallback(on_epoch_end=on_epoch_end_niet)


model1.fit(x, y,
          batch_size=128,
          epochs=EPOCHS,
          callbacks=[print_callback])


C:\Users\timjh\AppData\Local\Temp\ipykernel_1696\1356494578.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
C:\Users\timjh\AppData\Local\Temp\ipykernel_1696\1356494578.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), seqlen, len(chars)), dtype=n

Epoch 1/10
667/667 [==============================] - ETA: 0s - loss: 1.9682 - categorical_crossentropy: 1.9682 - accuracy: 0.4375
----- Generating text after Epoch: 0
667/667 [==============================] - 100s 147ms/step - loss: 1.9682 - categorical_crossentropy: 1.9682 - accuracy: 0.4375
Epoch 2/10
666/667 [============================>.] - ETA: 0s - loss: 1.5685 - categorical_crossentropy: 1.5685 - accuracy: 0.5384
----- Generating text after Epoch: 1
667/667 [==============================] - 95s 142ms/step - loss: 1.5684 - categorical_crossentropy: 1.5684 - accuracy: 0.5384
Epoch 3/10
666/667 [============================>.] - ETA: 0s - loss: 1.5072 - categorical_crossentropy: 1.5072 - accuracy: 0.5540
----- Generating text after Epoch: 2
667/667 [==============================] - 92s 138ms/step - loss: 1.5072 - categorical_crossentropy: 1.5072 - accuracy: 0.5540
Epoch 4/10
666/667 [============================>.] - ETA: 0s - loss: 1.4794 - categorical_crossentropy: 1.4794 - 

In [29]:
chars_date = sorted(list(set(text_date)))
char_indices = dict((c, i) for i, c in enumerate(chars_date))
indices_char = dict((i, c) for i, c in enumerate(chars_date))

seqlen = 40
step = seqlen
sentences = []
for i in range(40, len(text_date) - seqlen - 1, step):
    sentences.append(text_date[i: i + seqlen + 1])

x_d = np.zeros((len(sentences), seqlen, len(chars_date)), dtype=np.bool)
y_d = np.zeros((len(sentences), seqlen, len(chars_date)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x_d[i, t, char_indices[char_in]] = 1
        y_d[i, t, char_indices[char_out]] = 1

model2 = Sequential()
model2.add(LSTM(128, input_shape=(seqlen, len(chars_date)), return_sequences=True))
model2.add(Dense(len(chars_date), activation='softmax'))

model2.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def on_epoch_end_date(epoch, _):
    """Function invoked at end of each epoch."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text_date) - seqlen - 1)

    generated = ''
    sentence = text_date[start_index: start_index + seqlen]
    generated += sentence
    #print('----- Generating with seed: "' + sentence + '"')
    #sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars_date)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
        
        preds = model2.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

print_callback = LambdaCallback(on_epoch_end=on_epoch_end_date)

model2.fit(x_d, y_d,
          batch_size=128,
          epochs=EPOCHS,
          callbacks=[print_callback])

C:\Users\timjh\AppData\Local\Temp\ipykernel_1696\752095980.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x_d = np.zeros((len(sentences), seqlen, len(chars_date)), dtype=np.bool)
C:\Users\timjh\AppData\Local\Temp\ipykernel_1696\752095980.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_d = np.zeros((len(sentences), seqlen, len(chars_dat

Epoch 1/10
20/21 [===========================>..] - ETA: 0s - loss: 3.6126 - categorical_crossentropy: 3.6126 - accuracy: 0.1428
----- Generating text after Epoch: 0
21/21 [==============================] - 25s 1s/step - loss: 3.6080 - categorical_crossentropy: 3.6080 - accuracy: 0.1431
Epoch 2/10
20/21 [===========================>..] - ETA: 0s - loss: 3.2789 - categorical_crossentropy: 3.2789 - accuracy: 0.1816
----- Generating text after Epoch: 1
21/21 [==============================] - 23s 1s/step - loss: 3.2746 - categorical_crossentropy: 3.2746 - accuracy: 0.1822
Epoch 3/10
20/21 [===========================>..] - ETA: 0s - loss: 2.7622 - categorical_crossentropy: 2.7622 - accuracy: 0.2686
----- Generating text after Epoch: 2
21/21 [==============================] - 22s 1s/step - loss: 2.7591 - categorical_crossentropy: 2.7591 - accuracy: 0.2691
Epoch 4/10
20/21 [===========================>..] - ETA: 0s - loss: 2.4677 - categorical_crossentropy: 2.4677 - accuracy: 0.3227
----- G

In [40]:
print(np.shape(char_indices))

83


In [39]:
flat1 = Flatten()(model1.layers[-1].output)
flat2 = Flatten()(model2.layers[-1].output)
output = LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True)(flat1)

model3 = keras.Model(inputs=[model1.inputs,model2.inputs], outputs=output)

for i in range(len(model3.layers)-1):
    layer = model3.layers[i]
    layer.trainable = False

model3.add(Dense(len(chars), activation='softmax'))

def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    rand_evn = random.randint(0,100)

    generated = ''
    if rand_evn % 2 == 0:
        start_index = random.randint(0, len(text_date) - seqlen - 1)
        sentence = text_date[start_index: start_index + seqlen]
    else:
        start_index = random.randint(0, len(text_niet) - seqlen - 1)
        sentence = text_niet[start_index: start_index + seqlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
        
        preds = model3.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], 0.5)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
 
model3.fit(x, y,
          batch_size=128,
          epochs=EPOCHS,
          callbacks=[print_callback])

ValueError: Input 0 of layer "lstm_14" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 6480)